In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train.pkl')
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test.pkl')

In [0]:
train['ability_content']=train[train['event']==2]['event_contents'].apply(lambda x: x[8:]).apply(lambda x: x.split(';')[0])

In [4]:
ability_count =train.groupby(['species', 'ability_content'])['game_id'].count().reset_index(name='count')
ability_count

,species,ability_content,count
0,0.0,,834
1,0.0,AssaultMode,306
2,0.0,Attack,178
3,0.0,BattleMode,1151
4,0.0,BuildArmory,4438
...,...,...,...
385,2.0,SpawnInfestedTerran,1
386,2.0,SpawnLarva,130289
387,2.0,SprayZerg,101
388,2.0,Stop,6953


In [0]:
teran_top = ability_count[ability_count['species']==0].sort_values('count', ascending=False)['ability_content'][:10]
protoss_top = ability_count[ability_count['species']==1].sort_values('count', ascending=False)['ability_content'][:10]
zerg_top = ability_count[ability_count['species']==2].sort_values('count', ascending=False)['ability_content'][:10]

In [6]:
total_top = list()
total_top = list(teran_top.values) + list(protoss_top.values) + list(zerg_top.values)
len(teran_top), len(protoss_top), len(zerg_top), len(total_top)

(10, 10, 10, 30)

##플레이어 0만 추출

In [0]:
df_player0 = train[train['player']==0]
df_player0=df_player0[df_player0['ability_content'].isin(total_top) == True]

In [8]:
result = list()
for game_id, temp in tqdm(df_player0.groupby('game_id', sort=True)):
  ability_dict = {i:0 for i in total_top}
  ability_dict['game_id'] = game_id
  ability_dict.update(temp[['game_id', 'ability_content']].groupby('ability_content').count().T.to_dict('records')[0])
  result.append(ability_dict) 

100%|██████████| 38768/38768 [01:45<00:00, 366.10it/s]


In [0]:
pd.DataFrame(result).to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train_top50_0.csv')

##플레이어1만 추출

In [0]:
df_player1 = train[train['player']==1]
df_player1=df_player1[df_player1['ability_content'].isin(total_top) == True]

In [11]:
result = list()
for game_id, temp in tqdm(df_player1.groupby('game_id', sort=True)):
  ability_dict = {i:0 for i in total_top}
  ability_dict['game_id'] = game_id
  ability_dict.update(temp[['game_id', 'ability_content']].groupby('ability_content').count().T.to_dict('records')[0])
  result.append(ability_dict) 

100%|██████████| 38762/38762 [01:46<00:00, 364.24it/s]


In [0]:
pd.DataFrame(result).to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train_top50_1.csv')

# Player 0,1 merge

In [0]:
train_0= pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train_top50_0.csv')
train_1= pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/train_top50_1.csv')

train_0.drop('Unnamed: 0',axis=1,inplace=True)
train_1.drop('Unnamed: 0',axis=1,inplace=True)

In [0]:
train_merge=train_0.merge(train_1,on='game_id',how='outer')

In [0]:
train_merge['winner']=train.groupby('game_id')['winner'].min()

In [16]:
train_merge.head()

,TrainSCV_x,Attack_x,TrainMarine_x,BuildSupplyDepot_x,CalldownMULE_x,BuildRefinery_x,BuildBarracks_x,LowerSupplyDepot_x,UpgradeToOrbitalCommand_x,BuildCommandCenter_x,TrainProbe_x,BuildPylon_x,BuildGateway_x,BuildAssimilator_x,ChronoBoost_x,TrainAdept_x,BuildNexus_x,TransformToWarpGate_x,BuildCyberneticsCore_x,MorphDrone_x,SpawnLarva_x,MorphOverlord_x,MorphZergling_x,TrainQueen_x,BuildExtractor_x,CreepTumor_x,BuildHatchery_x,BuildCreepTumor_x,game_id,TrainSCV_y,Attack_y,TrainMarine_y,BuildSupplyDepot_y,CalldownMULE_y,BuildRefinery_y,BuildBarracks_y,LowerSupplyDepot_y,UpgradeToOrbitalCommand_y,BuildCommandCenter_y,TrainProbe_y,BuildPylon_y,BuildGateway_y,BuildAssimilator_y,ChronoBoost_y,TrainAdept_y,BuildNexus_y,TransformToWarpGate_y,BuildCyberneticsCore_y,MorphDrone_y,SpawnLarva_y,MorphOverlord_y,MorphZergling_y,TrainQueen_y,BuildExtractor_y,CreepTumor_y,BuildHatchery_y,BuildCreepTumor_y,winner
0,9.0,0.0,3.0,4.0,1.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,5.0,3.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,8.0,5.0,3.0,2.0,2.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.0,4.0,6.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,5.0,2.0,6.0,5.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,11.0,7.0,4.0,3.0,3.0,5.0,2.0,4.0,0.0
3,7.0,8.0,9.0,6.0,3.0,1.0,3.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,7.0,4.0,4.0,4.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14.0,14.0,3.0,2.0,3.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,3.0,4.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0


In [0]:
t_game_id=train_merge['game_id']
train_merge.drop('game_id',axis=1,inplace=True)

In [0]:
X=train_merge.iloc[:,:-1]
y=train_merge.iloc[:,-1]
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state=156)

# Test data 처리

In [0]:
test['ability_content']=test[test['event']==2]['event_contents'].apply(lambda x: x[8:]).apply(lambda x: x.split(';')[0])

In [0]:
df_player0 = test[test['player']==0]
df_player0=df_player0[df_player0['ability_content'].isin(total_top) == True]

df_player1 = test[test['player']==1]
df_player1=df_player0[df_player0['ability_content'].isin(total_top) == True]

In [21]:
result = list()
for game_id, temp in tqdm(df_player0.groupby('game_id', sort=True)):
  ability_dict = {i:0 for i in total_top}
  ability_dict['game_id'] = game_id
  ability_dict.update(temp[['game_id', 'ability_content']].groupby('ability_content').count().T.to_dict('records')[0])
  result.append(ability_dict) 

100%|██████████| 16739/16739 [00:46<00:00, 362.72it/s]


In [0]:
pd.DataFrame(result).to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test_top50_0.csv')
test_0= pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test_top50_0.csv')

In [23]:
result = list()
for game_id, temp in tqdm(df_player1.groupby('game_id', sort=True)):
  ability_dict = {i:0 for i in total_top}
  ability_dict['game_id'] = game_id
  ability_dict.update(temp[['game_id', 'ability_content']].groupby('ability_content').count().T.to_dict('records')[0])
  result.append(ability_dict) 

100%|██████████| 16739/16739 [00:46<00:00, 360.42it/s]


In [0]:
pd.DataFrame(result).to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test_top50_1.csv')
test_1= pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test_top50_1.csv')

In [0]:
test_0.drop('Unnamed: 0',axis=1,inplace=True)
test_1.drop('Unnamed: 0',axis=1,inplace=True)
test_merge=test_0.merge(test_1,on='game_id',how='outer')

ts_game_id=test_merge['game_id']
test_merge.drop('game_id',inplace=True,axis=1)

# Modeling

## Logistic Regression

In [26]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [28]:
#3. 모델 구성하기
model = Sequential()
model.add(Dense(256, input_dim=len(X.columns), activation ='relu'))
model.add(Dense(128, activation ='relu'))
model.add(Dense(64, activation ='relu'))
model.add(Dense(32, activation ='relu'))
model.add(Dense(16, activation ='relu'))
model.add(Dense(8, activation ='relu'))
model.add(Dense(1, activation ='sigmoid'))

#4. 모델 학습과정 설정하기
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m]) 



#5. 모델 학습시키기
#5.1. Early Stopping
callbacks_list = [
    EarlyStopping(
    monitor='val_loss',
    patience=10,
    ),
     ModelCheckpoint(
     filepath='multi_model.h5',
     monitor='val_loss',
     save_best_only=True,
     )
]

hist = model.fit(X_train, y_train, epochs = 100,callbacks=callbacks_list,validation_data=(X_val, y_val))

#6. 모델 평가하기
scores = model.evaluate(X_val, y_val)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

#epoch, loss, acc 그래프






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 27204 samples, validate on 11660 samples
Epoch 1/100





27204/27204 [==============================] - 13s 495us/step - loss: 0.6932 - acc: 0.4992 - f1_m: 0.3064 - precision_m: 0.2311 - recall_m: 0.4634 - val_loss: 0.6932 - val_acc: 0.4934 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/100
27204/27204 [==============================] - 4s 150us/step - loss: 0.6932 - acc: 0.4971 - f1_m: 0.2282 - precision_m: 0.1718 - recall_m: 0.3470 - val_loss: 0.6932 - val_acc: 0.4934 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/100
27204/27204 [==============================] - 4s 149us/step - loss: 0.6932 - acc: 0.5010 - f1_m: 0.2495 - precision_m: 0.1885 - recall_m: 0.3766 - val_loss: 0.6931 - val_acc: 0.5066 - val_f1_m: 0.6678 - val_precision_m: 0.5066 - val_recall_m: 1.0000
Epoch 4/100
27204/27204 [====

In [0]:
preds=model.predict(test_merge)

In [30]:
preds

array([[0.50056785],
       [0.50056785],
       [0.50056785],
       ...,
       [0.50056785],
       [0.50056785],
       [0.50056785]], dtype=float32)

In [0]:
#test_merge.drop('winner',axis=1,inplace=True)

In [0]:
test_merge['winner']=preds
test_merge.loc[test_merge['winner']>=0.5,'winner']=1
test_merge.loc[test_merge['winner']<0.5,'winner']=0

In [33]:
preds.shape

(16739, 1)

In [34]:
#16787
test_merge['winner'].value_counts()

1.0    16739
Name: winner, dtype: int64

In [0]:
#test_merge['game_id']=ts_game_id
#test_merge[['game_id','winner']].to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test_top_50.csv')